In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
# Random Forest
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error
from imblearn.over_sampling import SMOTE

In [2]:
# Import our 3-year modeling data
model_data = pd.read_csv('data/all_model_data.csv')
model_data

,DayOfMonth,DayOfWeek,DayOfYear,Month,Season_Autumn,Season_Spring,Season_Summer,Season_Winter,Year,cloudcover,...,sunrise_hour,sunset_hour,temp,tempmax,tempmin,uvindex,visibility,windgust,windspeed,CallsPerDay
0,4,0,4,1,0,0,0,1,2021,87,...,7,16,48,52,43,1,9,44,14,1
1,14,3,14,1,0,0,0,1,2021,49,...,7,16,44,53,38,3,9,11,8,29
2,14,3,14,1,0,0,0,1,2021,49,...,7,16,44,53,38,3,9,11,8,29
3,14,3,14,1,0,0,0,1,2021,49,...,7,16,44,53,38,3,9,11,8,29
4,14,3,14,1,0,0,0,1,2021,49,...,7,16,44,53,38,3,9,11,8,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66456,31,6,365,12,0,0,0,1,2023,100,...,7,16,41,42,39,1,0,5,5,87
66457,31,6,365,12,0,0,0,1,2023,100,...,7,16,41,42,39,1,0,5,5,87
66458,31,6,365,12,0,0,0,1,2023,100,...,7,16,41,42,39,1,0,5,5,87
66459,31,6,365,12,0,0,0,1,2023,100,...,7,16,41,42,39,1,0,5,5,87


In [3]:
# Import our upcoming prediction data
upcoming_data = pd.read_csv('data/upcoming_data_with_dummies.csv')
upcoming_data

,DayOfMonth,DayOfWeek,DayOfYear,Month,Season_Autumn,Season_Spring,Season_Summer,Season_Winter,Year,cloudcover,...,solarradiation,sunrise_hour,sunset_hour,temp,tempmax,tempmin,uvindex,visibility,windgust,windspeed
0,4,1,156,6,False,False,True,False,2024,79.1,...,91.9,5,20,60.2,69.0,53.0,4,10.1,16.1,6.9
1,5,2,157,6,False,False,True,False,2024,34.7,...,329.5,5,20,62.2,76.0,53.0,9,10.1,15.0,8.1
2,6,3,158,6,False,False,True,False,2024,4.0,...,357.1,5,20,64.6,80.0,49.0,10,10.1,16.1,8.1
3,7,4,159,6,False,False,True,False,2024,9.5,...,350.0,5,20,68.8,84.3,53.7,9,15.0,11.6,6.3
4,8,5,160,6,False,False,True,False,2024,40.8,...,323.9,5,20,67.1,78.9,54.9,9,15.0,13.2,7.2
5,9,6,161,6,False,False,True,False,2024,29.3,...,352.5,5,20,64.5,75.5,54.2,9,15.0,15.2,7.4
6,10,0,162,6,False,False,True,False,2024,7.2,...,365.1,5,20,64.2,76.7,51.5,10,15.0,15.0,7.4
7,11,1,163,6,False,False,True,False,2024,9.7,...,337.0,5,20,61.8,77.3,50.1,9,15.0,14.3,8.9


In [4]:
# Remove constant columns
constant_columns = model_data.columns[model_data.nunique() == 1]
model_data = model_data.drop(columns=constant_columns)

In [5]:
# List of columns to remove from both DataFrames
columns_to_drop = ['DayOfMonth', 'Year']
# Drop specified columns from model_data
model_data = model_data.drop(columns=columns_to_drop)
# Drop specified columns from upcoming_data
upcoming_data = upcoming_data.drop(columns=columns_to_drop)

In [6]:
# Find the number of unique rows
unique_rows = model_data.drop_duplicates()
unique_row_count = unique_rows.shape[0]

print(f'The number of unique rows is: {unique_row_count}')

The number of unique rows is: 1083


In [7]:
# Remove non-unique rows
model_data = model_data.drop_duplicates()

#### Finding Weights with OLS

In [8]:
# Split the data into features (X) and target variable (y)
X = model_data.drop(columns=['CallsPerDay'])
y = model_data['CallsPerDay']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train an OLS regression model
reg = LinearRegression()
reg.fit(X_train, y_train)

# Predict on the test set
y_pred = reg.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Print the weights (coefficients) of each feature, sorted by absolute value
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': reg.coef_
})

coefficients['AbsCoefficient'] = coefficients['Coefficient'].abs()
coefficients_sorted = coefficients.sort_values(by='AbsCoefficient', ascending=False).drop(columns='AbsCoefficient')

print("Coefficients sorted by absolute value:")
print(coefficients_sorted)

# Save the sorted coefficients to a CSV file
coefficients_sorted.to_csv('data/OLS_coefficients.csv', index=False)

# Check for data leakage
X_leakage = X_test[X_test.index.isin(X_train.index)]
if not X_leakage.empty:
    print("Warning: Data Leakage Detected!\n")

Mean Squared Error: 123.34807751523272
R-squared: -0.03976430910617945
Coefficients sorted by absolute value:
                                              Feature  Coefficient
12  conditions_Rain, Freezing Drizzle/Freezing Rai...    23.738971
30                                          snowdepth     9.668829
18            conditions_Snow, Rain, Partially cloudy    -9.062100
15                  conditions_Rain, Partially cloudy    -6.103032
16                              conditions_Snow, Rain     5.724238
14                          conditions_Rain, Overcast    -5.563554
11                                    conditions_Rain    -5.200094
13  conditions_Rain, Freezing Drizzle/Freezing Rai...    -5.087978
31                                        solarenergy     3.106629
33                                       sunrise_hour    -2.840731
6                                       Season_Winter    -2.581703
9                                 conditions_Overcast     2.464386
5                  

#### Risk Classification

In [9]:
# Convert the column names of both DataFrames to sets
model_data_columns = set(model_data.columns)
upcoming_data_columns = set(upcoming_data.columns)

# Columns in model_data but not in upcoming_data
model_only_columns = model_data_columns.difference(upcoming_data_columns)
print("Columns in model_data but not in upcoming_data:")
print(model_only_columns)

# Drop columns from model_data that are not in upcoming_data, except 'CallsPerDay'
columns_to_drop = [col for col in model_only_columns if col != 'CallsPerDay']
model_data = model_data.drop(columns=columns_to_drop)

# Columns in upcoming_data but not in model_data
upcoming_only_columns = upcoming_data_columns.difference(model_data_columns)
print("\nColumns in upcoming_data but not in model_data:")
print(upcoming_only_columns)

Columns in model_data but not in upcoming_data:
{'CallsPerDay'}

Columns in upcoming_data but not in model_data:
set()


In [10]:
# Find the 90th percentile for CallsPerDay
call_volume_threshold = model_data['CallsPerDay'].quantile(0.90)

# Label the days based on whether their call volume exceeds the threshold
model_data['HighRisk'] = (model_data['CallsPerDay'] >= call_volume_threshold).astype(int)

# Check for constant features
constant_features = [col for col in model_data.columns if model_data[col].nunique() <= 1]
if constant_features:
    print(f"Constant features found and will be dropped: {constant_features}")
    model_data = model_data.drop(columns=constant_features)

# Check feature correlation with the target
correlation = model_data.corr()['HighRisk'].abs().sort_values(ascending=False)
print("Feature correlation with target:")
print(correlation)

# Split the data into features (X) and target labels (y)
# Make sure 'CallsPerDay' is not included in the features
X = model_data.drop(columns=['CallsPerDay', 'HighRisk'])
y = model_data['HighRisk']

# Split the data into training and testing sets with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Handle class imbalance with SMOTE
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

# Print sizes of training and testing sets
print(f"Training set size after SMOTE: {X_train_sm.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")

# Increase the number of trees in the Random Forest classifier
n_trees = 100  # You can adjust this number to your needs

# Initialize and train a Random Forest classifier with cross-validation
clf = RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=n_trees)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(clf, X_train_sm, y_train_sm, cv=cv, scoring='accuracy')
print("Cross-validation accuracy scores:", cv_scores)
print("Mean cross-validation accuracy:", cv_scores.mean())

# Train the classifier on the training data
clf.fit(X_train_sm, y_train_sm)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Check label distribution
label_distribution = model_data['HighRisk'].value_counts(normalize=True)
print("Label Distribution:")
print(label_distribution)

Feature correlation with target:
HighRisk                                                             1.000000
CallsPerDay                                                          0.582356
conditions_Rain, Freezing Drizzle/Freezing Rain, Overcast            0.125404
temp                                                                 0.100431
tempmin                                                              0.099274
feelslikemin                                                         0.098176
feelslike                                                            0.094969
dew                                                                  0.090479
Season_Summer                                                        0.089393
tempmax                                                              0.083220
feelslikemax                                                         0.082454
sunrise_hour                                                         0.081306
snowdepth                      

In [11]:
# Save the feature data X to a CSV file
X.to_csv('features.csv', index=False)

In [12]:
# Use the trained classifier to predict on upcoming_data
upcoming_data_predictions = clf.predict(upcoming_data)

# Add the predictions as a new column to upcoming_data
upcoming_data['HighRisk'] = upcoming_data_predictions

# Print the number of high-risk days (1s) in the 'HighRisk' column of upcoming_data
num_high_risk_days = upcoming_data['HighRisk'].sum()
print(f"Number of high-risk days: {num_high_risk_days}")

Number of high-risk days: 2


In [13]:
# Save the DataFrame with predictions to a CSV file
upcoming_data.to_csv('data/upcoming_data_with_predictions.csv', index=False)

In [14]:
# Get feature importances
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': clf.feature_importances_})
feature_importances.sort_values(by='Importance', ascending=False, inplace=True)
print(feature_importances)

                                              Feature  Importance
24                                               pm25    0.062659
32                                     solarradiation    0.050070
38                                            uvindex    0.049229
40                                           windgust    0.048194
7                                          cloudcover    0.048053
1                                           DayOfYear    0.047541
23                                           humidity    0.046500
28                                   sealevelpressure    0.045618
22                                       feelslikemin    0.042976
31                                        solarenergy    0.042302
37                                            tempmin    0.041344
41                                          windspeed    0.037475
33                                       sunrise_hour    0.036317
36                                            tempmax    0.035450
35        